In [1]:
from Pipeline import PreProcessingPipeline
from DataSplit import DataSplit

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline as SparkPipeline
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from Config import Config
import os

/Users/julius/repos/BD-Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
preprocessing_pipeline = PreProcessingPipeline(DataSplit.TRAIN)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 10:24:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 10:24:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51202)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/Users/julius/repos/BD-Project/.venv/lib/python3.

In [3]:
df = preprocessing_pipeline.run()

24/03/14 10:24:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train-3.csv
24/03/14 10:24:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/

CodeCache: size=131072Kb used=23210Kb max_used=23210Kb free=107861Kb
 bounds [0x00000001079e0000, 0x00000001090b0000, 0x000000010f9e0000]
 total_blobs=9147 nmethods=8218 adapters=842
 compilation: disabled (not enough contiguous free space left)
Removed 0 duplicate rows.


In [4]:
df.show()

24/03/14 10:24:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train-6.csv


+---+----------+--------------+--------------------+----+-----------------+--------------------+--------------------+-----------+--------------+-------------------+-----+---------------+-------------+-------------+
| id|product_id|product_parent|       product_title|vine|verified_purchase|     review_headline|         review_body|review_date|marketplace_id|product_category_id|label|review_language|review_length|header_length|
+---+----------+--------------+--------------------+----+-----------------+--------------------+--------------------+-----------+--------------+-------------------+-----+---------------+-------------+-------------+
|  1|B00FAPF5U0|      46324555|    Candy Crush Saga|   N|                Y|brilliant but ann...|I took ages to si...| 2013-11-13|             1|                  1|False|             en|          160|           22|
|  4|0141319135|     871827197|Percy Jackson and...|   N|                N|                NULL|Sadly I am still ...| 2010-03-23|           

In [5]:
spark = SparkSession.builder.appName("RandomForestExample").getOrCreate()


24/03/14 10:24:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
# List of your feature columns
featureColumns = ["review_length", "header_length"]

# Assemble the feature columns into a single feature vector
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")


In [7]:
# If "label" is not a numeric column, convert it to numeric using StringIndexer
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)


In [8]:
(trainingData, testData) = df.randomSplit([0.7, 0.3]) # before we submit, we want to train on all training data though.


In [9]:
# Configure the Random Forest model
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="features", numTrees=10)

# Chain indexers and forest in a Pipeline
preprocessing_pipeline = SparkPipeline(stages=[labelIndexer, assembler, rf])

# Train model
model = preprocessing_pipeline.fit(trainingData)
# save the model
# model.save("model")


24/03/14 10:24:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/train-6.csv
24/03/14 10:24:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
 Schema: id, product_id, product_parent, product_title, vine, verified_purchase, review_headline, review_body, review_date, marketplace_id, product_category_id, label
Expected: id but found: 
CSV file: file:///Users/julius/repos/BD-Project/data/

In [10]:
if Config.MODEL_PATH in os.listdir():
    # delete model dir
    os.system(f"rm -r {Config.Model_PATH}")
model.save(Config.MODEL_PATH)

24/03/14 10:24:59 ERROR Instrumentation: java.io.IOException: Path model/ already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:174)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:169)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:42)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3$adapted(Pipeline.scala:344)
	at org.apache

Py4JJavaError: An error occurred while calling o543.save.
: java.io.IOException: Path model/ already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:174)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:169)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:42)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3$adapted(Pipeline.scala:

24/03/14 11:21:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1084147 ms exceeds timeout 120000 ms
24/03/14 11:21:25 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/14 11:21:26 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [ ]:
model = PipelineModel.load(Config.MODEL_PATH)

In [ ]:
predictions = model.transform(testData)

# Select example rows to display
predictions.select("prediction", "indexedLabel", "features").show(5)


In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="indexedLabel")
accuracy = evaluator.evaluate(predictions)
print(f"accuracy: {accuracy}")
